In [1]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_sm
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import json
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

import yaml
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/parser/embeddings/fasttext-sbwc.3.6.e20.vec'

In [2]:
cantidad = 300000

model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [3]:
path_to_json = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_seccionado'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)  # for me this prints ['foo.json']
jsons = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        jsons.append(json.load(json_file))
        


In [4]:
jsons[0]['experiencia']

'marketing gestión comercial persona proactiva ordenar alto sentir responsabilidad equipar fundamental empresa orientar resultar tangible experiencia laboral adecco peru cargo gerente desarrollo fecha 01/04/2019 fecho funciones responsable manejar cartero top cliente responsable cumplimiento objetivo venta líneo negociar garantizar rentabilidad nivel servicio desarrollo convenio diverso institución implementación canal post-venta cliente extranjero enfoque negociar cliente extranjero desarrollar servicio medir cliente adecco peru cargo gerente sucursal fecha 01/01/2017 31/03/2019 funciones responsable gestión integral sucursal responsable cumplimiento objetivo venta líneo negociar garantizar rentabilidad nivel servicio desarrollo ejecución plan negocio venta kpis estrategia comercial orientar captación mantenimiento retención cliente cargo ejecutivo comercial fecha 25/03/2015 31/12/2016 funciones ejecutar estrategia comercial orientar captación mantenimiento retención cliente sucursal 

In [5]:
def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [6]:
frase_1 = 'corredor de bolsa'
frase_2 = 'ingeniero comercial'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.3823823


In [7]:
print(model.most_similar(positive=['excel', 'sap'], negative=['datos']))

[('microsoft', 0.4660836458206177), ('xcode', 0.45749515295028687), ('barcode', 0.4524089992046356), ('tbarcode', 0.4520794153213501), ('oracle', 0.4513850212097168), ('quickbooks', 0.45035749673843384), ('koffice', 0.4440692663192749), ('kingsoft', 0.43898719549179077), ('iwork', 0.4389722943305969), ('delphi', 0.4341479241847992)]


In [8]:
model.similar_by_word('corredor', 20)

[('corredores', 0.6894433498382568),
 ('excorredor', 0.6477311849594116),
 ('corredoria', 0.5702216029167175),
 ('corredora', 0.5340113043785095),
 ('esprinter', 0.5244001150131226),
 ('velocista', 0.5186375975608826),
 ('ciclista', 0.5133304595947266),
 ('pasillo', 0.5123224258422852),
 ('corredoras', 0.4917184114456177),
 ('pedalista', 0.4917040169239044),
 ('esprint', 0.4876434803009033),
 ('esprinters', 0.4859365224838257),
 ('bioceánico', 0.4820821285247803),
 ('ciclistas', 0.4741329252719879),
 ('corridor', 0.47213947772979736),
 ('túnel', 0.4685114622116089),
 ('viario', 0.4659390449523926),
 ('tramo', 0.4606683850288391),
 ('trazado', 0.4597742259502411),
 ('contrarrelojista', 0.4591130018234253)]

In [9]:
get_closest(word= 'ingle', n = 3)

(['ingle', 'pantorrilla', 'pierna', 'rodilla'],
 [1, 0.733283281326294, 0.7193220853805542, 0.7045726776123047])

In [10]:
model.similarity('tradición', 'tradicional')

0.6669351

In [11]:
import es_core_news_md
nlp = es_core_news_md.load()
nlp_text = nlp('educado')
nlp_text[0].lemma_

'educar'

In [12]:
prc_description = '''ingeniería máster postgrado excel desarrollo gestión comercial experiencia manejo clientes
emprendimiento liderar equipos planificar organizar dirigir trabajo presión seguimiento KPI inglés '''

In [13]:
# https://github.com/prateekguptaiiitk/Resume_Filtering/blob/develop/Scoring/CV_ranking.ipynb
word_value = {}
similar_words_needed = 1
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        #print(similar_words[i], word_value[similar_words[i]])
        #print('------------------------------------------------')

In [14]:
word_value.keys()

dict_keys(['ingeniería', 'ingenierías', 'máster', 'másteres', 'postgrado', 'postgrados', 'excel', 'powerpoint', 'desarrollo', 'sostenible', 'gestión', 'gestionar', 'comercial', 'comerciales', 'experiencia', 'experiencial', 'manejo', 'manejar', 'clientes', 'cliente', 'emprendimiento', 'emprendimientos', 'liderar', 'encabezar', 'equipos', 'quipos', 'planificar', 'planear', 'organizar', 'organizarla', 'dirigir', 'dirigirlo', 'trabajo', 'remunerado', 'presión', 'presiones', 'seguimiento', 'seguimientos', 'kpi', 'krt', 'inglés', 'ingles'])

In [15]:
### ahora veamos si resulta el ranking
#frecuencia de término – frecuencia inversa de documento 
#Tf-idf
#Para calcular este ranking es mejor tener las secciones skills y experiencia
#con el fin de calcular esta metrica usando la ocurrencia de las palabras
# Tenemos todos los CV's y una descripción del cargo, a este descripcion del cargo
# tiene N palabras, le buscamos 2 palabras parecidas, generando una descripcion
# de N*2

# Usando esta nuevo set de palabras de descripcion, recorremos todos los cvs contando 
# la ocurrencia de estas palabras en cada documento, y luego se genera un ranking usando Tf-idf
# La pregunta es: ¿Lo haré sobre el documento entero? o ¿Trataré de seccionar y ocupar ciertas secciones?

no_of_cv = len(jsons)
#print(no_of_cv)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            if word in jsons[i]['skills'] or word in jsons[i]['experiencia'] or word in jsons[i]['educación']:
                #print('entre')
                count[word] += 1
        except:
            pass

    idf[word] = math.log((no_of_cv+1)/(1+count[word]))
    #print(idf)

In [16]:
#calculemos usando similitud



def calculo_similitud(word1, array_palabras):
    n_veces = 0
    for word in array_palabras:
        try:
            sim = model.similarity(word, word1)
            #print(word)
            #print(sim)
            if sim > 0.5:
                n_veces += 1
            else:
                continue
        except: #No estaba la palabra
            pass
        
    return n_veces



word1 = 'feliz'
array = ['alegre', 'contento', 'taladro', 'juguete']




n = calculo_similitud(word1, array)
n

2

In [17]:
jsons[0]['skills'].split()

[]

In [22]:
no_of_cv = len(jsons)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        # eliminación de stopwords y quizas lematizacion
        skill_pro = pre_process(jsons[i]['skills']) 
        expe_pro = pre_process(jsons[i]['experiencia'])
        edu_pro = pre_process(jsons[i]['educación'])
        
        if calculo_similitud(word, skill_pro.split()) or calculo_similitud(word, expe_pro.split()) or calculo_similitud(word, edu_pro.split()):
            count[word] += 1


    idf[word] = math.log((no_of_cv+1)/(1+count[word]))

FileNotFoundError: [Errno 2] No such file or directory: '/home/erwin/Genoma/cv-parser/parser/diccionarios/stop_words'

In [ ]:
idf

In [ ]:

score = {}
for i in range(no_of_cv):
    score[i] = 0
    skill_pro = pre_process(jsons[i]['skills']) 
    expe_pro = pre_process(jsons[i]['experiencia'])
    edu_pro = pre_process(jsons[i]['educación'])
    for word in word_value.keys():
        
        
        
        tf = 1 + calculo_similitud(word, skill_pro.split()) + calculo_similitud(word, expe_pro.split()) + calculo_similitud(word, edu_pro.split())

        score[i] += word_value[word]*tf*idf[word]

print(score)

In [ ]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], jsons[i]['nombre archivo']))
    
sorted_list.sort(reverse = True)


In [ ]:
sorted_list

In [ ]:
sorted_list

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
import spacy
import es_core_news_sm


def pre_process(corpus):
    corpus = corpus.lower()

    stopset = stopwords.words('spanish') + list(string.punctuation)

    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    corpus = unidecode(corpus)
    return corpus

def lematizar(frase):
    nlp = es_core_news_sm.load()
    doc = nlp(frase)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return lemmas



In [ ]:

test = pre_process('educación básica')
test


In [ ]:
test = lematizar('ingeniero')
test

In [ ]:
lematizar('ingeniero') in lematizar('ingenieria')

In [ ]:
model.similarity('ingeniero', 'ingenieria')

In [ ]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')
   # stemmed_claves = [stemmer.stem(token) for token in palabras_claves]
#stop_words = set(stopwords.words('spanish')) 

[stemmer.stem(x) for x in test] 
#stemmed_clave

In [ ]:
stemmer.stem('ingeniería') in stemmer.stem('ingeniería')

In [ ]:
import os
import nltk
import string
from nltk.tokenize import sent_tokenize, word_tokenize

def cargar_dict(path):
    with open(path) as f:  
        array = [x.strip() for x in f]
        c = [x for x in array if x != ''] # '' aparece cuando hay lines vacias
    return c

In [ ]:
newStopWords = cargar_dict('/home/erwin/Genoma/cv-parser/parser/diccionarios/stop_words')
stopwords = nltk.corpus.stopwords.words('spanish')
stopwords.extend(newStopWords)

In [ ]:



def pre_process(corpus, stopWords, enminiscula = True):
    if enminiscula:
        corpus = corpus.lower()
    stopset = stopwords+ list(string.punctuation)

    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    #corpus = unidecode.unidecode(corpus)
    return corpus

In [ ]:
pre_process('UNIVERSIDAD Universidad universidad', stopwords, False)

In [ ]:
import es_core_news_sm
def lematizar(frase):
    '''
    Esta función recibe un string y le aplica lematización:
    ingeniero ---> ingenier
    ingeniera ----> ingenier
    '''
    nlp = es_core_news_sm.load()
    doc = nlp(frase)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return lemmas

In [ ]:
des = '''
Nos encontramos en la búsqueda de Socios comerciales, para administrar por completo importantes puntos de ventas (corner) de retail de calzado.

Has escuchado hablar de las franquicias y consignaciones? pues esta empresa trabaja a través de la consignación, esto quiere decir que la empresa te hace entrega del punto de venta, corriendo con todos los gastos de infraestructura, arriendo y toda la mercadería y renovación de stock constante, apoyo de visual y marketing,etc.
Buscamos vendedores que se encarguen de la administración por completo del punto de venta (RRHH,seguimiento y gestión inventarios, bodegas, ventas, etc.). Ofrecemos atractivas comisiones

Los montos líquidos PROMEDIO mensual a los que puedes acceder, posterior a los gastos de administración van desde: $600.000 a $1.000.000

Requisitos:
-Experiencia en ventas (dependiente o independiente)
-Disponibilidad para horarios de mall
-Motivación y orientación al logro y a la venta
-Ambición y competitividad
-Manejo conceptos básicos de contabilidad y legislación laboral
-Excelente capacidad de administrar un equipo.
-Posee una sociedad limitada (si no cuenta con esto, no es impedimento, puesto que es tramite simple que se realiza de manera on line)


Buscamos personas que deseen emprender, tengan orientación a la venta y el liderazgo, para todos ellos esta es la oportunidad para iniciar un negocio propio con el respaldo de una marca líder en su rubro.
'''


f =lematizar(des)
lematizado = ''
for word in f:
    lematizado += word +' '
lematizado.replace('\n', '')

In [19]:
import os
import nltk
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 

def pre_process(corpus,  enminiscula= True):
    '''
    Entrada: texto, stopwords, enminiscula (opcional)
    Salida:  texto
    Funcion que se encarga de limpiar las stopwords de un texto
    el parámetro opciona enminuscula si es verdadero,
    transforma todo el texto a miniscula y elimina stopwords que esten en minuscula.
    Cuando se usa false, el texto retornado mantendra capitalizacion original y 
    además se eliminan stop words especificas tales como: Pontificia, Universidad, Vitae, VITAE
    Notar que stop_words.txt tiene stopwords en minisculas y capitalizada.
    Esta propiedad de mantener la capitalización es útil en la detección de nombres.
    '''
    newStopWords = cargar_dict('/home/erwin/Genoma/cv-parser/parser/diccionarios/stop_words')
    stopwords = nltk.corpus.stopwords.words('spanish')
    print(newStopWords)
    stop = stopwords.extend(newStopWords)
    

    if enminiscula:
        corpus = corpus.lower()
    stopset = stopwords+ list(string.punctuation)

    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    #corpus = unidecode.unidecode(corpus)
    return corpus

In [21]:
def cargar_dict(path):
    '''
    Utilidad para cargar los diccionarios
    '''
    with open(path) as f:  
        array = [x.strip() for x in f]
        c = [x for x in array if x != ''] # '' aparece cuando hay lines vacias
    return c

file = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1586903416737-NewHTJ'
cv_txt = cargar_dict(file)

In [23]:
def modificar(word):
    '''
     Retornar el lema de la palabra. 
   Fijandose que no sean un simbolo raro ni una stopword.
    '''
    try:
        symbols = '''~`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(mod_word)

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None # to handle the odd case of characters like 'x02', etc.
    

    

def esta_vacia(line):
    '''
    Retorna un booleano correspondiendo a 
    si una linea esta vacia en términos de letras-números
    '''
    for c in line:
        if (c.isalpha()):
            return False
    return True

In [ ]:
t = " ".join(cv_txt)


In [133]:
def secciones_limpio(dataframe):
    ar = [str(exp).lower() for exp in dataframe if str(exp)!='nan' and str(exp)!= ' ']
    return ar
    

In [142]:


path_secciones_dic = '/home/erwin/Genoma/cv-parser/parser/CSVs/Secciones CVs_buscador.csv'

secciones_dic = pd.read_csv(path_secciones_dic)
secciones_dic.head()

,Perfil,Experiencia,Educacion,Cursos,Habilidades,Contacto,Referencias,Logros,Hobbies,Emprendimientos
0,Perfil,Experiencia Laboral,Formacion Academica,Cursos complementarios,Aptitudes,Datos personales,Referencias laborales,Logros,Hobbies,Emprendimientos
1,Resumen,Experiencia Profesional,Educacion,Cursos,Conocimientos,Contacto,Referencias,Logros y objetivos alcanzados,"Hobbies, actividades y deportes",NaN
2,Biografía Profesional,Antecedentes Laborales,Formacion,Cursos y seminarios,Habilidades,Detalles personales,Referencias personales,Metas profesionales,Interes,NaN
3,Sobre mi,Experiencia,Estudios,Talleres,Skills,Informacion de contacto,Mis referencias,NaN,Areas de interes,NaN
4,Acerca de mi,Experiencias,Formacion educativa,Seminarios,Cualificaciones,Otros antecedentes,NaN,NaN,Miscelaneo,NaN


In [156]:
experiencia = secciones_limpio(secciones_dic.Experiencia)
perfil = secciones_limpio(secciones_dic.Perfil)
educacion = secciones_limpio(secciones_dic.Educacion)
cursos = secciones_limpio(secciones_dic.Cursos)
habilidades = secciones_limpio(secciones_dic.Habilidades) 
contacto = secciones_limpio(secciones_dic.Contacto)
referencias = secciones_limpio(secciones_dic.Referencias)
logros = secciones_limpio(secciones_dic.Logros)
hobbies = secciones_limpio(secciones_dic.Hobbies)

otros = perfil + educacion + cursos + habilidades + contacto + referencias + logros+ hobbies
#otros

In [175]:
def extraer_experiencia(cv_text):
    linea_experiencia = False
    siguiente_seccion = False
    parrafo = ''
    for line in cv_txt:
        line_np = re.sub(r'[^\w\s]','', line)
        l = sum([i.strip(string.punctuation).isalpha() for i in line_np.split()])
        chunks = re.split(' +', line)
        linea =''
        for word in chunks:
            linea += word.lower() + ' '

        if ((len(line.strip()) == 0 or l > 4) and linea_experiencia == False):
            continue


        linea = " ".join(linea.split())
        for experiencia in experiencia_list:
            linea_np = re.sub(r'[^\w\s]','', linea)
            experiencia_np = re.sub(r'[^\w\s]','', experiencia)
            linea_un = "".join(unidecode.unidecode(linea_np).split())
            experiencia_un = "".join(unidecode.unidecode(experiencia_np).split())
            if experiencia_un.lower() == linea_un.lower():
                #print('He pillado la seccion')
                linea_experiencia = True
                #continue


        for otro in otros:
            otro_np = re.sub(r'[^\w\s]','', otro)
            linea_np = re.sub(r'[^\w\s]','', linea)
            otro_un = unidecode.unidecode(otro_np)
            linea_un = unidecode.unidecode(linea_np)

            if linea_un.lower() == otro_un.lower() and linea_experiencia:
                #print(linea_un.upper())
                siguiente_seccion = True
                break

        if siguiente_seccion:
            break

        if linea_experiencia == True and siguiente_seccion == False:
            parrafo += linea + '\n'

    return parrafo

In [176]:
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1586786838778-CV_Postulaciones'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1571410211245-CV_Mariana_Wong'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1571275460067-CV_Gonzalo_Vásquez'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1569272625085-CV_Arianne.R.'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1568522376573-CV_Jorge_Berna_Espinoza'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1569025536095-19.08.19_CV_Nicolas_Achondo'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1568672819667-CV_CAMILO_BUSTAMANTE_SANTANDERR'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1566924682475-CV_CatalinaZunigaBilbao'
path = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1566746473048-Patricio_Mendez-CV'
file = path
cv_txt = open(file, "r")

extraer_experiencia(cv_txt)

'experiencia laboral\nde 21/01/2019 a la fecha\nminera antucoya, superintendencia de geotecnia\ncargo ocupado practicante 21/01/2019 01/03/2019\n\nactualización continua de base de datos estructurales, mediante mapeos de bancos y\ndatos i-site.\n\ncreación de script para software vulcan, a fin de realizar actualizaciones a modelos de\ncorto plazo con unidades geotécnicas.\n\ncreación de plantas del modelo de corto plazo, con unidades geotécnicas y\ndensidades, para área de perforación y tronadura.\n\nanálisis de potenciales mecanismos de inestabilidad en software tangram de bancos\nextraídos y bancos por extraer.\n\ncreación de superficies compuestas diseño de fase avance de mina para análisis\nde mecanismos de inestabilidad en bancos por extraer.\ncargo ocupado memorista 01/03/2019 actualidad\n\nactualización de estructuras mayores e intermedias, a partir de mapeos de bancos y\ndatos complementarios i-site obtenidos en escaneos de alta resolución\n\nmodelamiento de superficies de fall

In [289]:
file = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1566924682475-CV_CatalinaZunigaBilbao'
cv_txt = open(file, "r").read()
#print(cv_txt)
def extraer_perfil(cv_text):
    otros = educacion + cursos + habilidades + contacto + referencias + logros+ hobbies + experiencia
    n = -1
    siguiente_seccion = False
    parrafo = ''
    #print(cv_text)
    #text_1 = cv_text
    n_linea = 0
    for line in cv_text.splitlines():
        n += 1
        #print(line)
        for resumen in perfil:
            linea_np = re.sub(r'[^\w\s]','', line)
            resumen_np = re.sub(r'[^\w\s]','', resumen)
            linea_un = "".join(unidecode.unidecode(linea_np).split())
            resumen_un = "".join(unidecode.unidecode(resumen_np).split())
            if resumen_un.lower() == linea_un.lower():
                linea_resumen = True
                print('pille linea resumen')
                print(resumen_un)
                n_linea = n
                break
    print(n_linea)
    #print(cv_txt)   
    for line in cv_text.splitlines()[n_linea:-1]:   
        #print('entre aca')
        chunks = re.split(' +', line)
        linea =''
        for word in chunks:
            linea += word.lower() + ' '

        linea = " ".join(linea.split())



        for otro in otros:
            otro_np = re.sub(r'[^\w\s]','', otro)
            linea_np = re.sub(r'[^\w\s]','', linea)
            otro_un = unidecode.unidecode(otro_np)
            linea_un = unidecode.unidecode(linea_np)

            if linea_un.lower() == otro_un.lower():
                #print(linea_un.upper())
                siguiente_seccion = True
                break

        if siguiente_seccion:
            break

        if  siguiente_seccion == False:
            parrafo += linea + '\n'

    return parrafo
    

    

In [301]:


experiencia = secciones_limpio(secciones_dic.Experiencia)
perfil = secciones_limpio(secciones_dic.Perfil)
educacion = secciones_limpio(secciones_dic.Educacion)
cursos = secciones_limpio(secciones_dic.Cursos)
habilidades = secciones_limpio(secciones_dic.Habilidades) 
contacto = secciones_limpio(secciones_dic.Contacto)
referencias = secciones_limpio(secciones_dic.Referencias)
logros = secciones_limpio(secciones_dic.Logros)
hobbies = secciones_limpio(secciones_dic.Hobbies)

otros = perfil + educacion + cursos + habilidades + contacto +  logros+ hobbies + experiencia

In [335]:

file = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1566924682475-CV_CatalinaZunigaBilbao'
cv_txt = open(file, "r").read()

def extraer_referencias(cv_text):
    linea_referencia = False
    siguiente_seccion = False
    parrafo = ''
    for line in cv_txt.splitlines():
        line_np = re.sub(r'[^\w\s]','', line)
        l = sum([i.strip(string.punctuation).isalpha() for i in line_np.split()])
        chunks = re.split(' +', line)
        linea =''
        for word in chunks:
            linea += word.lower() + ' '

        if ((len(line.strip()) == 0 or l > 4) and linea_referencia == False):
            continue

        #print(linea + '\n')
        linea = " ".join(linea.split())

        for referencia in referencias:
            linea_np = re.sub(r'[^\w\s]','', linea)
            referencia_np = re.sub(r'[^\w\s]','', referencia)
            linea_un = "".join(unidecode.unidecode(linea_np).split())
            referencia_un = "".join(unidecode.unidecode(referencia_np).split())
            if referencia_un.lower() == linea_un.lower():
                print('He pillado la seccion')
                linea_referencia = True
                #print(linea.UPPER())
                #continue


        for otro in otros:
            otro_np = re.sub(r'[^\w\s]','', otro)
            linea_np = re.sub(r'[^\w\s]','', linea)
            otro_un = unidecode.unidecode(otro_np)
            linea_un = unidecode.unidecode(linea_np)

            if linea_un.lower() == otro_un.lower() and linea_referencia:
                print(linea_un)
                siguiente_seccion = True
                break

        if siguiente_seccion:
            break

        if linea_referencia == True and siguiente_seccion == False:
            parrafo += linea + '\n'
            
    if len(parrafo.splitlines())>1:
        parrafo = "\n ".join([str(x) for x in parrafo.splitlines()[1:-1]])
        

    return parrafo


In [336]:
print(extraer_referencias(cv_txt))

He pillado la seccion
maría cristina vallejos. líder zona centro sur productividad,
 vicepresidencia de proyectos, codelco- 975496775
 victor encina. jefe de carrera ingeniería civil de minas utfsm-
 98875257
 milka casanegra. presidenta rim, ingeniera tronadura qa/qc


In [337]:
print(cv_txt)

 Experiencia profesional
2018-2019
Red de ingenieras de Minas de Chile-RIM
Cargo
Fundadora
Función
Planificación, coordinación y
ejecución de actividades
relacionadas con el empoderamiento femenino y la inclusión de
género en la industria minera. Meet and Greet, Charla
habilidades blandas, Reuniones con empresas generando
alianzas
2019
Triconos Mineros
Cargo
Memorista
Función
Estudio y análisis de la información en el área de perforación de
Minera Los Pelambres, con el objetivo de elaborar un plan de
mejoramiento de la gestión de la calidad de la información.
Feb. 2018
Nueva Ancortecmin S.a
Cargo Practica Industrial
Función
Seguimiento de costos y presupuestos con el objetivo de
levantar aprendizajes y oportunidades de mejora.
Enero 2014 Endesa S.a
Cargo Practica Industrial
Función
Desarrollo de tareas relacionadas con el control automático e
instrumentación de plantas termoeléctricas, revisando planos y
correcta ejecución de funcionamiento de sistemas.
Catalina
Zúñiga
Bilbao
Avenida T